## 1️⃣ Clone & Setup

In [3]:
# Clone repository
!git clone https://github.com/Usernamenisiya/thesis-cloud-rl.git
%cd thesis-cloud-rl

# Verify
!pwd
!ls -la | head -15

Cloning into 'thesis-cloud-rl'...
remote: Enumerating objects: 1765, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 1765 (delta 8), reused 16 (delta 6), pack-reused 1742 (from 2)
Receiving objects: 100% (1765/1765), 654.03 MiB | 16.78 MiB/s, done.
Resolving deltas: 100% (244/244), done.
Updating files: 100% (2632/2632), done.
/content/thesis-cloud-rl/thesis-cloud-rl
/content/thesis-cloud-rl/thesis-cloud-rl
total 40064
drwxr-xr-x 4 root root     4096 Jan 12 14:24 .
drwxr-xr-x 5 root root     4096 Jan 12 14:24 ..
-rw-r--r-- 1 root root     3567 Jan 12 14:24 analyze_data_distribution.py
-rw-r--r-- 1 root root     5945 Jan 12 14:24 cloudsen12_loader.py
-rw-r--r-- 1 root root     1661 Jan 12 14:24 cnn_inference.py
-rw-r--r-- 1 root root   200302 Jan 12 14:24 colab_training.ipynb
drwxr-xr-x 3 root root     4096 Jan 12 14:24 data
-rw-r--r-- 1 root root     4697 Jan 12 14:24 data_download.py
-rw-r--r-- 1 root root     4276 Jan

In [4]:
# Install dependencies
!pip install -r requirements.txt
!pip install gymnasium  # Updated from deprecated gym

import torch
import stable_baselines3
import rasterio

print("✅ Dependencies installed")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

✅ Dependencies installed
PyTorch: 2.9.0+cu126
CUDA available: True


In [5]:
# Check GPU
!nvidia-smi

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n✅ Using device: {device}")

Mon Jan 12 14:25:40 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             42W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 2️⃣ Setup CloudSEN12 Real Ground Truth Data

**Using CloudSEN12 expert-labeled dataset:**
- Already downloaded: 100 patches in `Colab_Data/cloudsen12_subset/`
- Process with `cloudsen12_loader.py` to extract 10 bands
- **26M pixels** for robust evaluation

In [6]:
# Mount Google Drive
from google.colab import drive
import os
from pathlib import Path

drive.mount('/content/drive')

# Verify CloudSEN12 data exists
cloudsen_path = '/content/drive/MyDrive/Colab_Data/cloudsen12_subset'

if os.path.exists(cloudsen_path):
    num_patches = len([d for d in Path(cloudsen_path).iterdir() if d.is_dir()])
    print(f"✅ CloudSEN12 data found: {num_patches} patches")
    print(f"📂 Location: {cloudsen_path}")

    # Process CloudSEN12 data with loader (extracts 10 bands, converts masks)
    print("\n🔧 Processing CloudSEN12 patches...")
    !python cloudsen12_loader.py
else:
    print(f"❌ CloudSEN12 data not found at: {cloudsen_path}")
    print("Please run CloudSEN12 download notebook first")

Mounted at /content/drive
✅ CloudSEN12 data found: 100 patches
📂 Location: /content/drive/MyDrive/Colab_Data/cloudsen12_subset

🔧 Processing CloudSEN12 patches...

🔧 Preparing CloudSEN12 for Training
📦 Loading CloudSEN12 Data

✅ Found 100 patches to load


✅ Successfully loaded 100 patches
📊 Image shape: (512, 512, 10)
📊 Mask shape: (512, 512)
📊 Image bands: 10
📊 Cloud coverage: 16.0%

💾 Saving 100 patches to data/cloudsen12_processed
/usr/local/lib/python3.12/dist-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
  Saved: patch_000
  Saved: patch_001
  Saved: patch_002
  Saved: patch_003
  Saved: patch_004
  Saved: patch_005
  Saved: patch_006
  Saved: patch_007
  Saved: patch_008
  Saved: patch_009
  Saved: patch_010
  Saved: patch_011
  Saved: patch_012
  Saved: patch_013
  Saved: patch_014
  Saved: patch_015
  Saved: patch_016
  Saved: patch_017
  Saved: patch_018
  Saved

In [7]:
# Verify processed CloudSEN12 data
import os
from pathlib import Path
import glob

processed_dir = 'data/cloudsen12_processed'
image_files = glob.glob(f'{processed_dir}/*_image.tif')
mask_files = glob.glob(f'{processed_dir}/*_mask.tif')

if len(image_files) > 0 and len(mask_files) > 0:
    print(f"✅ CloudSEN12 data processed successfully!")
    print(f"📊 Found {len(image_files)} image patches")
    print(f"📊 Found {len(mask_files)} mask patches")
    print("\n🎯 Ready for training with real ground truth!")
else:
    print("❌ Processed data not found")
    print("Please check cloudsen12_loader.py output for errors")

✅ CloudSEN12 data processed successfully!
📊 Found 100 image patches
📊 Found 100 mask patches

🎯 Ready for training with real ground truth!


## 3️⃣ Check CNN Baseline

In [9]:
# Load and test CNN baseline on CloudSEN12 patches
from cnn_inference import load_sentinel2_image, get_cloud_mask
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import rasterio
import numpy as np
import glob

print("🧠 Evaluating CNN Baseline on CloudSEN12 Real Ground Truth")
print("="*60)

# Load all processed CloudSEN12 patches
image_files = sorted(glob.glob('data/cloudsen12_processed/*_image.tif'))
mask_files = sorted(glob.glob('data/cloudsen12_processed/*_mask.tif'))

all_gt = []
all_cnn = []

print(f"Processing {len(image_files)} patches...\n")

for img_path, mask_path in zip(image_files, mask_files):  # Use ALL patches
    # Load image and get CNN prediction
    image = load_sentinel2_image(img_path)
    cnn_prob = get_cloud_mask(image)

    # Load real ground truth
    with rasterio.open(mask_path) as src:
        ground_truth = src.read(1)

    # Binary conversion
    gt_binary = (ground_truth > 0).astype(np.uint8)
    cnn_binary = (cnn_prob > 0.5).astype(np.uint8)

    all_gt.append(gt_binary.flatten())
    all_cnn.append(cnn_binary.flatten())

# Combine all patches
all_gt = np.concatenate(all_gt)
all_cnn = np.concatenate(all_cnn)

# Calculate metrics
accuracy = accuracy_score(all_gt, all_cnn)
precision = precision_score(all_gt, all_cnn, zero_division=0)
recall = recall_score(all_gt, all_cnn, zero_division=0)
f1 = f1_score(all_gt, all_cnn, zero_division=0)

print(f"\n📊 Evaluated on {len(image_files)} CloudSEN12 patches")
print(f"📊 Total pixels: {len(all_gt):,}")
print("\n🧠 CNN Baseline (Real Ground Truth):")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")

print(f"  F1-Score:  {f1:.4f}")
print(f"📊 CNN predicted: {all_cnn.sum():,} cloud pixels ({all_cnn.mean()*100:.1f}%)")
print(f"\n📊 Ground truth: {all_gt.sum():,} cloud pixels ({all_gt.mean()*100:.1f}%)")

🧠 Evaluating CNN Baseline on CloudSEN12 Real Ground Truth
Processing 100 patches...



/usr/local/lib/python3.12/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)



📊 Evaluated on 100 CloudSEN12 patches
📊 Total pixels: 26,214,400

🧠 CNN Baseline (Real Ground Truth):
  Accuracy:  0.6652
  Precision: 0.1313
  Recall:    0.1935
  F1-Score:  0.1564
📊 CNN predicted: 6,198,343 cloud pixels (23.6%)

📊 Ground truth: 4,205,740 cloud pixels (16.0%)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## 4️⃣ Pull Latest Code & Train PPO

In [10]:
# Get latest code with PPO improvements
!git pull origin master
print("✅ Repository updated")

From https://github.com/Usernamenisiya/thesis-cloud-rl
 * branch            master     -> FETCH_HEAD
Already up to date.
✅ Repository updated


## 🎯 Comprehensive Approach: Three Methods

We'll implement three approaches with progressive improvements:

1. **Optimal Threshold (Classical)** - Grid search, 5 minutes
2. **CNN Fine-Tuning (Transfer Learning)** - Domain adaptation, 30 minutes  
3. **RL Threshold Refinement (Novel)** - Spatially-adaptive thresholds, 1 hour

This provides:
- ✅ Multiple baselines for comparison
- ✅ Progressive improvement narrative
- ✅ Novel RL contribution that actually improves results

### 📊 Approach 1: Optimal Threshold (Grid Search)

In [ ]:
# Optimal threshold grid search (fast, no training)
print("🔍 Finding optimal CNN threshold via grid search...")
print("Testing thresholds from 0.1 to 0.9 on train set")
print("="*60)

!python optimize_threshold_grid_search.py

### 📊 Approach 2: Optimal Threshold Evaluated on Test Set

In [ ]:
# Evaluate optimal threshold from train set on test set
print("📊 Evaluating optimal threshold on test set...")
print("Applying best threshold from training to held-out test data")
print("="*60)

!python finetune_cnn_cloudsen12.py

### 🎯 Approach 3: RL Adaptive Threshold Refinement (Novel Contribution)

In [ ]:
# RL-based adaptive threshold refinement (1 hour)
print("🎯 Training RL agent for spatially-adaptive thresholds...")
print("Agent learns to adjust CNN threshold per patch based on local context")
print("Action: continuous threshold delta [-0.3, +0.3]")
print("Reward: F1-score improvement over baseline")
print("="*60)

!python train_ppo_threshold_refinement.py

### 📊 Compare All Approaches

In [ ]:
# Compare all three approaches
import json
from pathlib import Path

print("\n" + "="*60)
print("📊 COMPREHENSIVE COMPARISON - ALL APPROACHES")
print("="*60)

# Load baseline CNN (threshold=0.5)
baseline_f1 = 0.2571  # From earlier evaluation
baseline_acc = 0.6719

print("\n🧠 Baseline CNN (threshold=0.5):")
print(f"  Accuracy:  {baseline_acc:.4f}")
print(f"  F1-Score:  {baseline_f1:.4f}")

# Load optimal threshold results (TEST SET)
if Path('results/optimal_threshold_test_results.json').exists():
    with open('results/optimal_threshold_test_results.json') as f:
        opt_results = json.load(f)
    opt_threshold = opt_results['optimal_threshold']
    opt_f1 = opt_results['test_optimal_metrics']['f1_score']
    opt_acc = opt_results['test_optimal_metrics']['accuracy']
    
    print(f"\n📊 Approach 1: Optimal Threshold ({opt_threshold:.2f}):")
    print(f"  Accuracy:  {opt_acc:.4f}")
    print(f"  F1-Score:  {opt_f1:.4f}")
    print(f"  Improvement: {(opt_f1 - baseline_f1) / baseline_f1 * 100:+.2f}%")

# Load RL threshold refinement results
if Path('results/threshold_refinement_results.json').exists():
    with open('results/threshold_refinement_results.json') as f:
        rl_results = json.load(f)
    rl_f1 = rl_results['rl_threshold_refinement']['f1_score']
    rl_acc = rl_results['rl_threshold_refinement']['accuracy']
    
    print(f"\n🎯 Approach 2: RL Adaptive Threshold:")
    print(f"  Accuracy:  {rl_acc:.4f}")
    print(f"  F1-Score:  {rl_f1:.4f}")
    print(f"  Improvement: {(rl_f1 - baseline_f1) / baseline_f1 * 100:+.2f}%")
    
    mean_delta = rl_results['threshold_statistics']['mean_delta']
    print(f"  Mean threshold adjustment: {mean_delta:+.4f}")

print("\n" + "="*60)
print("✅ All approaches evaluated!")
print("="*60)

### 📸 Visual Comparison

In [ ]:
# Visualize predictions from all three approaches on test patches
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import glob
from pathlib import Path
from stable_baselines3 import PPO
from cnn_inference import load_sentinel2_image, get_cloud_mask
from rl_threshold_environment import ThresholdRefinementEnv

# Load test patches
data_dir = 'data/cloudsen12_processed'
image_files = sorted(glob.glob(f'{data_dir}/*_image.tif'))
mask_files = sorted(glob.glob(f'{data_dir}/*_mask.tif'))

split_idx = int(0.8 * len(image_files))
test_image_files = image_files[split_idx:]
test_mask_files = mask_files[split_idx:]

# Select 3 diverse test patches to visualize
patch_indices = [0, 10, 19]  # First, middle, last

# Load RL model
model_path = sorted(glob.glob('models/ppo_threshold_refinement_*'))[-1]
rl_model = PPO.load(model_path)
print(f"✅ Loaded RL model: {Path(model_path).name}")

# Optimal threshold from results
with open('results/optimal_threshold_test_results.json') as f:
    opt_threshold = json.load(f)['optimal_threshold']

print(f"📊 Comparing: Baseline (0.5) | Optimal ({opt_threshold:.2f}) | RL Adaptive\n")

# Create visualization
fig, axes = plt.subplots(len(patch_indices), 5, figsize=(20, 4*len(patch_indices)))

for row, patch_idx in enumerate(patch_indices):
    img_path = test_image_files[patch_idx]
    mask_path = test_mask_files[patch_idx]
    
    # Load data
    image = load_sentinel2_image(img_path)
    cnn_prob = get_cloud_mask(image)
    
    with rasterio.open(mask_path) as src:
        ground_truth = src.read(1)
    
    # Get predictions
    baseline_pred = (cnn_prob > 0.5).astype(np.uint8)
    optimal_pred = (cnn_prob > opt_threshold).astype(np.uint8)
    
    # RL adaptive prediction
    env = ThresholdRefinementEnv(image, cnn_prob, ground_truth, patch_size=64, baseline_threshold=0.5)
    rl_pred = np.zeros_like(ground_truth, dtype=np.uint8)
    
    obs, _ = env.reset()
    for _ in range(env.num_patches):
        i, j = env.current_pos
        action, _ = rl_model.predict(obs, deterministic=True)
        threshold_delta = np.clip(action[0], -0.3, 0.3)
        adjusted_threshold = np.clip(0.5 + threshold_delta, 0.1, 0.9)
        
        cnn_patch = cnn_prob[i:i+64, j:j+64]
        rl_pred[i:i+64, j:j+64] = (cnn_patch > adjusted_threshold).astype(np.uint8)
        
        obs, _, done, _, _ = env.step(action)
        if done:
            break
    
    # Ground truth
    gt_binary = (ground_truth > 0).astype(np.uint8)
    
    # RGB visualization (bands 4,3,2 for true color)
    rgb = image[:, :, [3, 2, 1]]  # B04, B03, B02
    rgb = np.clip(rgb / 3000, 0, 1)  # Normalize
    
    # Plot
    axes[row, 0].imshow(rgb)
    axes[row, 0].set_title(f'Patch {patch_idx+1}\nRGB Image')
    axes[row, 0].axis('off')
    
    axes[row, 1].imshow(gt_binary, cmap='gray', vmin=0, vmax=1)
    axes[row, 1].set_title('Ground Truth')
    axes[row, 1].axis('off')
    
    axes[row, 2].imshow(baseline_pred, cmap='gray', vmin=0, vmax=1)
    axes[row, 2].set_title(f'Baseline (0.5)\nF1: {f1_score(gt_binary.flatten(), baseline_pred.flatten()):.3f}')
    axes[row, 2].axis('off')
    
    axes[row, 3].imshow(optimal_pred, cmap='gray', vmin=0, vmax=1)
    axes[row, 3].set_title(f'Optimal ({opt_threshold:.2f})\nF1: {f1_score(gt_binary.flatten(), optimal_pred.flatten()):.3f}')
    axes[row, 3].axis('off')
    
    axes[row, 4].imshow(rl_pred, cmap='gray', vmin=0, vmax=1)
    axes[row, 4].set_title(f'RL Adaptive\nF1: {f1_score(gt_binary.flatten(), rl_pred.flatten()):.3f}')
    axes[row, 4].axis('off')

plt.tight_layout()
plt.savefig('results/comparison_visualization.png', dpi=150, bbox_inches='tight')
print(f"\n💾 Visualization saved to: results/comparison_visualization.png")
plt.show()